# Oppmerksomhetsmekanismer og transformatorer

En stor ulempe med rekurrente nettverk er at alle ord i en sekvens har samme innvirkning på resultatet. Dette fører til suboptimal ytelse med standard LSTM-encoder-decoder-modeller for sekvens-til-sekvens-oppgaver, som for eksempel navngitt enhetsgjenkjenning og maskinoversettelse. I virkeligheten har spesifikke ord i inngangssekvensen ofte større innvirkning på sekvensielle utganger enn andre.

Tenk på en sekvens-til-sekvens-modell, som maskinoversettelse. Den implementeres ved hjelp av to rekurrente nettverk, der ett nettverk (**encoder**) komprimerer inngangssekvensen til en skjult tilstand, og et annet nettverk, **decoder**, ruller ut denne skjulte tilstanden til et oversatt resultat. Problemet med denne tilnærmingen er at den endelige tilstanden til nettverket vil ha vanskeligheter med å huske begynnelsen av en setning, noe som fører til dårlig modellkvalitet for lange setninger.

**Oppmerksomhetsmekanismer** gir en måte å vekte den kontekstuelle innvirkningen av hver inngangsvektor på hver utgangsprediksjon av RNN. Dette implementeres ved å lage snarveier mellom mellomliggende tilstander i inngangs-RNN og utgangs-RNN. På denne måten, når vi genererer utgangssymbolet $y_t$, tar vi hensyn til alle skjulte inngangstilstander $h_i$, med forskjellige vektkoeffisienter $\alpha_{t,i}$.

![Bilde som viser en encoder/decoder-modell med et additivt oppmerksomhetslag](../../../../../translated_images/no/encoder-decoder-attention.7a726296894fb567.webp)
*Encoder-decoder-modellen med additiv oppmerksomhetsmekanisme i [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), sitert fra [denne bloggposten](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Oppmerksomhetsmatrisen $\{\alpha_{i,j}\}$ representerer graden til hvilken visse inngangsord spiller en rolle i genereringen av et gitt ord i utgangssekvensen. Nedenfor er et eksempel på en slik matrise:

![Bilde som viser en eksempeljustering funnet av RNNsearch-50, hentet fra Bahdanau - arviz.org](../../../../../translated_images/no/bahdanau-fig3.09ba2d37f202a6af.webp)

*Figur hentet fra [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Oppmerksomhetsmekanismer er ansvarlige for mye av dagens eller nesten dagens toppytelse innen naturlig språkbehandling. Å legge til oppmerksomhet øker imidlertid antall modellparametere betydelig, noe som førte til skaleringsproblemer med RNN-er. En viktig begrensning ved å skalere RNN-er er at modellens rekurrente natur gjør det utfordrende å batch-prosessere og parallellisere trening. I en RNN må hvert element i en sekvens behandles i sekvensiell rekkefølge, noe som betyr at det ikke enkelt kan parallelliseres.

Adopsjonen av oppmerksomhetsmekanismer kombinert med denne begrensningen førte til opprettelsen av de nåværende toppmoderne transformator-modellene som vi kjenner og bruker i dag, fra BERT til OpenGPT3.

## Transformator-modeller

I stedet for å føre konteksten fra hver tidligere prediksjon inn i neste evalueringssteg, bruker **transformator-modeller** **posisjonelle kodinger** og oppmerksomhet for å fange konteksten til en gitt inngang innenfor et gitt tekstvindu. Bildet nedenfor viser hvordan posisjonelle kodinger med oppmerksomhet kan fange kontekst innenfor et gitt vindu.

![Animasjon som viser hvordan evalueringene utføres i transformator-modeller.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Siden hver inngangsposisjon kartlegges uavhengig til hver utgangsposisjon, kan transformatorer parallellisere bedre enn RNN-er, noe som muliggjør mye større og mer uttrykksfulle språkmodeller. Hver oppmerksomhetshode kan brukes til å lære forskjellige relasjoner mellom ord som forbedrer oppgaver innen naturlig språkbehandling.

**BERT** (Bidirectional Encoder Representations from Transformers) er et svært stort flerlags transformatornettverk med 12 lag for *BERT-base* og 24 for *BERT-large*. Modellen blir først forhåndstrent på en stor tekstkorpus (Wikipedia + bøker) ved hjelp av usupervisert trening (predikere maskerte ord i en setning). Under forhåndstreningen absorberer modellen et betydelig nivå av språkforståelse som deretter kan utnyttes med andre datasett ved hjelp av finjustering. Denne prosessen kalles **overføringslæring**.

![Bilde fra http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/no/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Det finnes mange varianter av transformator-arkitekturer, inkludert BERT, DistilBERT, BigBird, OpenGPT3 og flere, som kan finjusteres. [HuggingFace-pakken](https://github.com/huggingface/) gir et bibliotek for å trene mange av disse arkitekturene med PyTorch.

## Bruke BERT til tekstklassifisering

La oss se hvordan vi kan bruke en forhåndstrent BERT-modell for å løse vår tradisjonelle oppgave: sekvensklassifisering. Vi skal klassifisere vårt originale AG News-datasett.

Først laster vi inn HuggingFace-biblioteket og datasettet vårt:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Fordi vi skal bruke en forhåndstrent BERT-modell, må vi bruke en spesifikk tokenizer. Først laster vi inn en tokenizer som er knyttet til den forhåndstrente BERT-modellen.

HuggingFace-biblioteket inneholder et arkiv med forhåndstrente modeller, som du kan bruke ved å spesifisere navnene deres som argumenter til `from_pretrained`-funksjoner. Alle nødvendige binærfiler for modellen vil automatisk bli lastet ned.

Imidlertid vil du noen ganger trenge å laste inn dine egne modeller. I slike tilfeller kan du spesifisere katalogen som inneholder alle relevante filer, inkludert parametere for tokenizer, `config.json`-filen med modellparametere, binære vekter, osv.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer`-objektet inneholder `encode`-funksjonen som kan brukes direkte til å kode tekst:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Da la oss lage iteratorer som vi vil bruke under trening for å få tilgang til dataene. Fordi BERT bruker sin egen kodingsfunksjon, må vi definere en utfyllingsfunksjon som ligner på `padify` vi har definert tidligere:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

I vårt tilfelle vil vi bruke en forhåndstrent BERT-modell kalt `bert-base-uncased`. La oss laste inn modellen ved hjelp av pakken `BertForSequenceClassification`. Dette sikrer at modellen vår allerede har en nødvendig arkitektur for klassifisering, inkludert den endelige klassifikatoren. Du vil se en advarsel som sier at vektene til den endelige klassifikatoren ikke er initialisert, og at modellen vil kreve forhåndstrening - det er helt greit, fordi det er akkurat det vi skal gjøre!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Nå er vi klare til å begynne treningen! Siden BERT allerede er forhåndstrent, ønsker vi å starte med en ganske lav læringsrate for å unngå å ødelegge de opprinnelige vektene.

Alt det harde arbeidet utføres av `BertForSequenceClassification`-modellen. Når vi kjører modellen på treningsdataene, returnerer den både tap og nettverksutgang for minibatch-inndata. Vi bruker tapet for parameteroptimalisering (`loss.backward()` utfører bakoverpasset), og `out` for å beregne treningsnøyaktighet ved å sammenligne de oppnådde etikettene `labs` (beregnet ved hjelp av `argmax`) med forventede `labels`.

For å kontrollere prosessen akkumulerer vi tap og nøyaktighet over flere iterasjoner, og skriver dem ut hver `report_freq` treningssyklus.

Denne treningen vil sannsynligvis ta ganske lang tid, så vi begrenser antall iterasjoner.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Du kan se (spesielt hvis du øker antall iterasjoner og venter lenge nok) at BERT-klassifisering gir oss ganske god nøyaktighet! Dette er fordi BERT allerede forstår språkstrukturen ganske godt, og vi trenger bare å finjustere den endelige klassifikatoren. Men siden BERT er en stor modell, tar hele treningsprosessen lang tid og krever betydelig datakraft! (GPU, og helst mer enn én).

> **Merk:** I vårt eksempel har vi brukt en av de minste forhåndstrente BERT-modellene. Det finnes større modeller som sannsynligvis vil gi bedre resultater.


## Evaluere modellens ytelse

Nå kan vi evaluere ytelsen til modellen vår på testdatasettet. Evalueringsløkken ligner ganske mye på treningsløkken, men vi må ikke glemme å sette modellen i evalueringsmodus ved å kalle `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Viktige punkter

I denne enheten har vi sett hvor enkelt det er å ta en forhåndstrent språkmodell fra **transformers**-biblioteket og tilpasse den til vår tekstklassifiseringsoppgave. På samme måte kan BERT-modeller brukes til enhetsuttrekking, spørsmål-svar og andre NLP-oppgaver.

Transformer-modeller representerer dagens toppmoderne teknologi innen NLP, og i de fleste tilfeller bør dette være den første løsningen du eksperimenterer med når du implementerer skreddersydde NLP-løsninger. Likevel er det svært viktig å forstå de grunnleggende prinsippene for rekurrente nevrale nettverk som ble diskutert i denne modulen, dersom du ønsker å bygge avanserte nevrale modeller.



---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
